In [7]:
import pandas as pd
import numpy as np
import re

In [8]:
print("A carregar o dataset original")
try:
    df = pd.read_csv("Hotel_Reviews.csv")
    print(f"Dataset Original carregado: {df.shape[0]} linhas, {df.shape[1]} colunas")
except FileNotFoundError:
    print("ERRO: O ficheiro 'Hotel_Reviews.csv' não foi encontrado. Verifica a pasta.")

A carregar o dataset original
Dataset Original carregado: 515738 linhas, 17 colunas


In [9]:
print("A limpar dados ruidosos")

# O Booking.com preenche reviews vazias com "No Negative" ou "No Positive".
# Isto confunde a IA (ela acha que a palavra "Negative" está no texto).
# Substituímos por string vazia.
df["Negative_Review"] = df["Negative_Review"].replace("No Negative", "")
df["Positive_Review"] = df["Positive_Review"].replace("No Positive", "")

# Substituir valores vazios reais (NaN) por string vazia
df["Negative_Review"] = df["Negative_Review"].fillna("")
df["Positive_Review"] = df["Positive_Review"].fillna("")

print("Limpeza de textos padrão concluída.")

A limpar dados ruidosos
Limpeza de textos padrão concluída.


In [10]:
def processar_texto(row):
    """
    Combina as reviews positiva e negativa e adiciona contexto (Nome do Hotel e Endereço).
    Não removemos stopwords para manter a semântica para modelos Transformer.
    """
    # Juntar as partes positiva e negativa
    review_texto = (str(row["Negative_Review"]) + " " + str(row["Positive_Review"])).strip()
    
    # Se, depois de limpar, a review estiver vazia, retorna vazio (será removida depois)
    if not review_texto:
        return np.nan
    
    # ESTRATÉGIA RAG: Injetar Metadados no Texto
    # O modelo de busca vai ler isto tudo junto. 
    # Ajuda a encontrar "Hotel em Londres" porque a palavra "London" passa a fazer parte do texto vetorial.
    texto_final = (
        f"Hotel Name: {row['Hotel_Name']}. "
        f"Location: {row['Hotel_Address']}. "
        f"Review: {review_texto}"
    )
    
    # Limpeza Leve (apenas remover espaços duplos e quebras de linha)
    texto_final = " ".join(texto_final.split())
    
    return texto_final

print("Função de processamento definida.")

Função de processamento definida.


In [11]:
print("A criar a coluna de Texto Enriquecido")

# Aplica a função a todas as linhas do DataFrame
df['review'] = df.apply(processar_texto, axis=1)

# Remove linhas que ficaram vazias depois do processamento (reviews sem texto útil)
df = df.dropna(subset=['review'])

print(f"Processamento concluído. Linhas válidas restantes: {len(df)}")

A criar a coluna de Texto Enriquecido
Processamento concluído. Linhas válidas restantes: 515501


In [ ]:
import pandas as pd
import numpy as np

TAMANHO_FINAL = 40000 

print(f"A iniciar redução inteligente de {len(df)} linhas...")

# 1. FILTRO DE QUALIDADE (Riqueza Semântica)
# Mantemos apenas reviews com mais de 15 palavras
df['word_count'] = df['review'].apply(lambda x: len(str(x).split()))
df_rico = df[df['word_count'] >= 15].copy()

print(f"   - Reviews curtas removidas. Restam: {len(df_rico)}")

# 2. AMOSTRAGEM ESTRATIFICADA (Balanceamento Geográfico)
df_rico['Country'] = df_rico['Hotel_Address'].apply(lambda x: x.split()[-1])

paises = df_rico['Country'].unique()
linhas_por_pais = int(TAMANHO_FINAL / len(paises))

dfs_temp = []
print(f"A recolher aprox. {linhas_por_pais} reviews por país...")

for pais in paises:
    df_pais = df_rico[df_rico['Country'] == pais]
    if len(df_pais) > linhas_por_pais:
        df_pais = df_pais.sample(n=linhas_por_pais, random_state=42)
    dfs_temp.append(df_pais)

df_final = pd.concat(dfs_temp).sample(frac=1, random_state=42).reset_index(drop=True)

print(f"Redução concluída. Temos {len(df_final)} linhas prontas para o NLP.")

📊 A iniciar redução inteligente de 515501 linhas...
   - Reviews curtas removidas. Restam: 515332
   - A recolher aprox. 4166 reviews por país...
Redução concluída. Temos 24996 linhas prontas para o NLP.


In [ ]:
import re
from collections import Counter

print("A extrair keywords nas reviews selecionadas...")

STOPWORDS = {
    'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 
    'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 
    'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 
    'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 
    'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 
    'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 
    'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 
    'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 
    'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 
    'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 
    'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 
    'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'now', 'd', 
    'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', 'couldn', 'didn', 'doesn', 'hadn', 
    'hasn', 'haven', 'isn', 'ma', 'mightn', 'mustn', 'needn', 'shan', 'shouldn', 'wasn', 
    'weren', 'won', 'wouldn', 'hotel', 'room', 'staff', 'stay', 'location', 'would', 'could'
}

def extrair_keywords_robusto(texto):
    if pd.isna(texto) or texto == "":
        return ""
    palavras = re.findall(r'\b[a-zA-Z]{3,}\b', str(texto).lower())
    palavras_uteis = [p for p in palavras if p not in STOPWORDS]
    contagem = Counter(palavras_uteis)
    top_5 = [palavra for palavra, freq in contagem.most_common(5)]
    return ", ".join(top_5)

df_final['keywords_pln'] = df_final['review'].apply(extrair_keywords_robusto)

colunas_uteis = [
    'Hotel_Name', 
    'Hotel_Address', 
    'review', 
    'Negative_Review', 
    'Positive_Review',
    'keywords_pln',  
    'Average_Score', 
    'Tags',
    'lat', 
    'lng'
]

df_final = df_final[colunas_uteis]

print(f"A guardar ficheiro final...")
df_final.to_csv("Hotel_Reviews_processed.csv", index=False)
print("Csv criado")

A aplicar PLN: Extração de Keywords
Extração concluída! Exemplo da 1ª linha: made, via, booking, booked, next
